In [1]:
import os
import shutil
import sys

import json

import numpy as np
from scipy import sparse
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sn
sn.set()

import pandas as pd
import implicit

from tqdm.notebook import tqdm

### Load data

In [2]:
# Just choose the name of the dataset directory
dataset  = 'ml-20m'
DATA_DIR = '/Users/tomas/Documents/FEUP/Tese/data/' + dataset
PARSE_DATA_DIR = os.path.join(DATA_DIR, 'processed_70_10_20')

In [3]:
unique_sid = list()
with open(os.path.join(PARSE_DATA_DIR, 'unique_sid.txt'), 'r') as f:
    for line in f:
        unique_sid.append(line.strip())

n_items = len(unique_sid)

In [4]:
def load_csv_data_to_sparse(csv_file):
    tp = pd.read_csv(csv_file)
    n_users = tp['uid'].max() + 1
    rows, cols = tp['uid'], tp['sid']
    data = sparse.csr_matrix((np.ones_like(rows),
                             (rows, cols)), dtype='float64',
                             shape=(n_users, n_items))
    return data, n_users

In [5]:
#TRAIN
#user - items
train_data, n_users = load_csv_data_to_sparse(os.path.join(PARSE_DATA_DIR, 'train.csv'))

In [6]:
train_data.shape

(129757, 11518)

In [7]:
#VALIDATION
#user - items
validation_data = pd.read_csv(os.path.join(PARSE_DATA_DIR, 'validation.csv'))

In [8]:
#TEST
#user - items
test_data = pd.read_csv(os.path.join(PARSE_DATA_DIR, 'test.csv'))

In [285]:
t = pd.read_csv(os.path.join(PARSE_DATA_DIR, 'train.csv'))
df = t.groupby('uid').size().reset_index(name='cont')
df = df.drop('uid', 1)
df['colFromIndex'] = df.index
df = df.sort_values(by=['cont', 'colFromIndex'], ascending=True)
df.head()

,cont,colFromIndex
33,6,33
140,6,140
219,6,219
257,6,257
276,6,276


In [286]:
t.head()

,uid,sid
0,0,997
1,0,2700
2,0,3619
3,0,144
4,0,1273


### Evaluation

In [9]:
def get_score(eval_df, pred, k, algo):
    """Compute the metrics for all the users in pred and compares it with the eval_df. 
    Args:
        eval_df: pandas df with interaction format (uid,sid). It will be used to compare the with the prediction  
        pred: numpy array with the predicions for each user
        k: Number of results to consider
        algo: Name of the algorithm
        
    Returns:
        Discounted cumulative gain
    """
    
    algo = algo + "_ndcg"
    n_users = eval_df['uid'].max() + 1
    grouped = eval_df.groupby('uid')

    pbar = tqdm()
    pbar.reset(total=n_users)
    


    df = pd.DataFrame(columns=['uid', algo]) 

    for i, (_, group) in enumerate(grouped):

        relevance = 1*np.isin(pred[i],group['sid'])
        ndcg = ndcg_at_k(relevance, k)
        df = df.append({'uid': i,algo: ndcg }, ignore_index=True)
        pbar.update()


    pbar.refresh()
    print("--- ", algo, " STATS ---")
    print("mean: ", df[algo].mean())
    print("max: ", df[algo].max())
    print("min: ", df[algo].min())
    return df

In [10]:
def dcg_at_k(r, k, method=0):
    """Score is discounted cumulative gain (dcg)
    Relevance is positive real values.  Can use binary
    as the previous methods.
    Example from
    http://www.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf
    >>> r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
    >>> dcg_at_k(r, 1)
    3.0
    >>> dcg_at_k(r, 1, method=1)
    3.0
    >>> dcg_at_k(r, 2)
    5.0
    >>> dcg_at_k(r, 2, method=1)
    4.2618595071429155
    >>> dcg_at_k(r, 10)
    9.6051177391888114
    >>> dcg_at_k(r, 11)
    9.6051177391888114
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]
    Returns:
        Discounted cumulative gain
    """
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.
 

In [11]:
def ndcg_at_k(r, k=100, method=0):
    """Score is normalized discounted cumulative gain (ndcg)
    Relevance is positive real values.  Can use binary
    as the previous methods.
    Example from
    http://www.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf
    >>> r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
    >>> ndcg_at_k(r, 1)
    1.0
    >>> r = [2, 1, 2, 0]
    >>> ndcg_at_k(r, 4)
    0.9203032077642922
    >>> ndcg_at_k(r, 4, method=1)
    0.96519546960144276
    >>> ndcg_at_k([0], 1)
    0.0
    >>> ndcg_at_k([1], 2)
    1.0
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]
    Returns:
        Normalized discounted cumulative gain
    """
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max or dcg_max == 0:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

## Modelling

### ALS

In [12]:
NUM_THREADS = 0
ALS_TOP_N = 20
ALS_Factors = 200

In [13]:
als_model = implicit.als.AlternatingLeastSquares(factors=ALS_Factors,calculate_training_loss=True)

In [14]:
als_model.fit(train_data.T)

In [31]:
als_factors = pd.DataFrame.from_records(als_model.user_factors)
als_factors['uid'] = validation_data.uid.unique()
als_factors.to_csv(os.path.join(PARSE_DATA_DIR, 'als_factors.csv'), index=False)

In [44]:
als_factors.head()

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,uid
0,-0.058967,0.229568,-0.707001,0.540223,-0.110984,0.219160,0.191912,0.397641,0.412974,0.055434,...,-0.166100,0.179952,0.182467,0.175274,-0.002350,0.520412,0.183931,-0.089415,0.294081,0
1,0.053650,0.003824,-0.151874,0.223005,-0.076169,0.024204,-0.226282,-0.275735,-0.287022,0.155725,...,0.015414,0.253139,0.023921,-0.157192,0.178530,0.095084,-0.079894,-0.121272,-0.002467,1
2,-0.202143,-0.226526,-0.648742,0.234206,0.170624,0.496507,-0.282112,0.455193,-0.109085,0.303810,...,-0.321206,0.287338,-0.166969,0.110853,0.312051,0.262411,0.061551,0.050908,-0.211964,2
3,-0.155596,0.016257,0.255050,0.119575,-0.315419,-0.087295,-0.286379,-0.054195,-0.147150,0.054961,...,-0.055110,-0.010709,0.287566,0.097178,0.251507,-0.101567,0.070255,0.004031,-0.131053,3
4,-0.017234,0.420561,-0.288330,0.445641,-0.164420,0.055104,-0.175972,-0.130913,-0.084127,0.221143,...,0.161235,0.038432,0.312712,-0.041273,0.328645,0.034584,0.055259,-0.273009,-0.147063,4


In [35]:
with open(os.path.join(PARSE_DATA_DIR, 'map_user.json')) as f:
    map_user = json.load(f)
    map_user_dict = {'original_id': list(map_user.keys()), 'mapped_id':list(map_user.values()) }
    map_user_df = pd.DataFrame(map_user_dict)

In [36]:
map_user_df.head()

,original_id,mapped_id
0,1,0
1,2,1
2,3,2
3,4,3
4,5,4


In [47]:
results_metadataset = pd.merge(left=map_user_df, right=als_factors, left_on='mapped_id', right_on='uid')
results_metadataset = results_metadataset.drop('mapped_id', 1)

In [52]:
ALS_DIR = os.path.join(PARSE_DATA_DIR, 'embeddings/als')
results_metadataset.to_csv(os.path.join(ALS_DIR, 'metadataset.csv'), index=False)

In [305]:
als_pred = als_model.recommend_all(user_items=train_data, N=ALS_TOP_N)

In [306]:
als_results_test = get_score(validation_data,als_pred,ALS_TOP_N, 'als')

---  als_ndcg  STATS ---
mean:  0.2054566410258193
max:  1.0
min:  0.0


In [240]:
als_results_test = get_score(test_data,als_pred,ALS_TOP_N, 'als')

---  als_ndcg  STATS ---
mean:  0.18506398346742625
max:  1.0
min:  0.0


## Comparison

| Algorithm | Parameters | NDCG TEST mean |
| --- | --- | --- |
| ALS | TOP_N = 20 Factors = 100 | 0.17689 |


### Bayesian Personalized Ranking

In [322]:
BPR_TOP_N = 20
BPR_Factors = 50

In [323]:
model_bpr = implicit.bpr.BayesianPersonalizedRanking(factors=BPR_Factors)

In [324]:
model_bpr.fit(train_data.T)

In [325]:
pred_bpr = model_bpr.recommend_all(user_items=train_data, N=BPR_TOP_N)

In [326]:
get_score(validation_data, pred_bpr, BPR_TOP_N, 'bpr')

---  bpr_ndcg  STATS ---
mean:  0.16877694415016598
max:  1.0
min:  0.0


,uid,bpr_ndcg
0,0.0,1.000000
1,1.0,0.000000
2,2.0,0.270238
3,3.0,0.000000
4,4.0,0.306138
5,5.0,0.244651
6,6.0,0.000000
7,7.0,0.250000
8,8.0,0.000000
9,9.0,0.000000


In [245]:
results_test_bpr = get_score(test_data, pred_bpr, BPR_TOP_N, 'bpr')

---  bpr_ndcg  STATS ---
mean:  0.14272910280252418
max:  1.0
min:  0.0


In [246]:
total_results =  pd.merge(als_results_test, results_test_bpr, on='uid')

In [247]:
total_results.head()

,uid,als_ndcg,bpr_ndcg
0,0.0,0.00000,0.0
1,1.0,0.63093,0.0
2,2.0,0.00000,0.0
3,3.0,0.00000,0.0
4,4.0,0.30103,0.0


### Logistic Matrix Factorization

In [347]:
LMF_TOP_N = 20
LMF_FACTORS = 75
#we are going to use the standard factors

In [348]:
model_LMF = implicit.lmf.LogisticMatrixFactorization(factors=LMF_FACTORS)

In [349]:
model_LMF.fit(train_data.T)

In [350]:
pred_LMF = model_LMF.recommend_all(user_items=train_data, N=LMF_TOP_N)

In [351]:
results_val_lmf = get_score(validation_data, pred_LMF, LMF_TOP_N, 'lmf')

---  lmf_ndcg  STATS ---
mean:  0.11152139707779593
max:  1.0
min:  0.0


In [252]:
results_test_lmf = get_score(test_data, pred_LMF, LMF_TOP_N, 'lmf')

---  lmf_ndcg  STATS ---
mean:  0.11216736487728235
max:  1.0
min:  0.0


In [253]:
total_results = pd.merge(total_results, results_test_lmf, on='uid')

In [254]:
total_results.head()

,uid,als_ndcg,bpr_ndcg,lmf_ndcg
0,0.0,0.00000,0.0,0.00000
1,1.0,0.63093,0.0,0.00000
2,2.0,0.00000,0.0,0.46598
3,3.0,0.00000,0.0,0.00000
4,4.0,0.30103,0.0,0.00000


### Baseline Most Popular

In [255]:
most_popular_TOP_N = 10

In [256]:
def get_most_popular(train_data, k=100):
    """Compute recommendations for all users based on the baseline most popular  
        Args:
            train_data: user_items matrix. it will be used to filter the items already clicked  
            pred: numpy array with the predicions for each user
            k: Number of results to consider

        Returns:
        np.array with the recommendations for all users
    """
    print('-- Start --')
    train_array = np.array(train_data.toarray())
    print('-- converted train_data to np.array --')
    
    #get the item popularity saved in a json file
    with open(os.path.join(PARSE_DATA_DIR, 'map_item_pop.json')) as f:
        map_item_pop = json.load(f)
    print('-- read json with item popularity --')
    
    #parse map_item_pop to dataframe may be useful, get the popularity as a np.array  
    df_pop = pd.DataFrame(list(map_item_pop.items()),columns = ['sid','pop'])
    item_popularity_np = np.array(df_pop['pop'])
    print('-- created np.array with the item popularity --')

    #invert the sparse matrix, in this way we can clean which items did a user already clicked
    not_train_array = 1*np.logical_not(train_array)
    print('-- inverted train_np_array --')

    #Compute a np array per user with the amount of each item was clicked 
    #in this way we can sort and with the indices get the most popular for each user
    clicks_item_per_user = np.multiply(not_train_array,item_popularity_np)
    not_train_array = None
    item_popularity_np = None
    train_array = None    
    print('-- multiplied inverted array * item popularity --')


    #prepare variables
    recommendations_list = []
    n_users = clicks_item_per_user.shape[0]
    
    print('-- gets the recommendations for all users --')
    #start progress bar
    pbar = tqdm()
    pbar.reset(total=n_users)

    for i in range(0,n_users):
        # for each user get the most clicked s_id 
        sorted_indices_array = np.argsort(clicks_item_per_user[i])
        top_k = sorted_indices_array[-k:]
        recommendations_list.append(top_k[::-1])
        pbar.update()


    recommendations_np = np.array(recommendations_list)
    pbar.refresh()
    print('-- Im finished master --')
    return recommendations_np

In [257]:
pred_most_popular = get_most_popular(train_data=train_data, k=most_popular_TOP_N)

-- Start --
-- converted train_data to np.array --
-- read json with item popularity --
-- created np.array with the item popularity --
-- inverted train_np_array --
-- multiplied inverted array * item popularity --
-- gets the recommendations for all users --


-- Im finished master --


In [258]:
results_test_most_popular = get_score(test_data, pred_most_popular, most_popular_TOP_N, 'most_popular')

---  most_popular_ndcg  STATS ---
mean:  0.14947415146428333
max:  1.0
min:  0.0


In [259]:
total_results = pd.merge(total_results, results_test_most_popular, on='uid')

In [260]:
total_results.head(10)

,uid,als_ndcg,bpr_ndcg,lmf_ndcg,most_popular_ndcg
0,0.0,0.000000,0.000000,0.000000,0.000000
1,1.0,0.630930,0.000000,0.000000,0.315465
2,2.0,0.000000,0.000000,0.465980,0.000000
3,3.0,0.000000,0.000000,0.000000,0.000000
4,4.0,0.301030,0.000000,0.000000,0.344770
5,5.0,0.000000,1.000000,0.356207,0.000000
6,6.0,0.657732,0.000000,1.000000,0.000000
7,7.0,0.630930,0.430677,0.333333,0.000000
8,8.0,0.301030,0.630930,0.443426,0.000000
9,9.0,0.000000,0.000000,0.000000,0.000000


### KNN Item-Item

In [261]:
knn_K = 10
cos = implicit.nearest_neighbours.CosineRecommender(K=knn_K, num_threads=NUM_THREADS)
tfidf = implicit.nearest_neighbours.TFIDFRecommender(K=knn_K, num_threads=NUM_THREADS)
bm25 = implicit.nearest_neighbours.BM25Recommender(K=knn_K, num_threads=NUM_THREADS)

models = [cos, tfidf, bm25]

In [262]:
for model in models:
    model.fit(train_data.T)

In [263]:
def compute_knn_recomendations(train_data, model, TOP_N=100):
    recommendations_list = []
    n_users = train_data.shape[0]

    pbar = tqdm()
    pbar.reset(total=n_users)
    for i in range(0,n_users):
        user_recomendations_tuples = model.recommend(i,train_data, N=TOP_N)
        user_recomendations_sid = [t[0] for t in user_recomendations_tuples]
        recommendations_list.append(user_recomendations_sid)
        pbar.update()

    pbar.refresh()
    recomendations_np = np.array(recommendations_list)
    return recomendations_np

In [264]:
pbar = tqdm()
pbar.reset(total=len(models))

for model, name in zip(models, ['knn_cos', 'knn_tfidf', 'knn_bm25']):
    pred_knn = compute_knn_recomendations(train_data, model, knn_K)
    results_knn = get_score(test_data, pred_knn, knn_K, name)
    print("model finished")
    pbar.update()
    total_results = pd.merge(total_results, results_knn, on='uid')
    
pbar.refresh()

---  knn_cos_ndcg  STATS ---
mean:  0.1928497727073329
max:  1.0
min:  0.0
model finished


---  knn_tfidf_ndcg  STATS ---
mean:  0.1875900462247455
max:  1.0
min:  0.0
model finished


---  knn_bm25_ndcg  STATS ---
mean:  0.1304619268104152
max:  1.0
min:  0.0
model finished


True

In [265]:
total_results.head()

,uid,als_ndcg,bpr_ndcg,lmf_ndcg,most_popular_ndcg,knn_cos_ndcg,knn_tfidf_ndcg,knn_bm25_ndcg
0,0.0,0.00000,0.0,0.00000,0.000000,0.000000,0.000000,0.315465
1,1.0,0.63093,0.0,0.00000,0.315465,0.666667,0.416667,0.000000
2,2.0,0.00000,0.0,0.46598,0.000000,0.000000,0.000000,0.430677
3,3.0,0.00000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000
4,4.0,0.30103,0.0,0.00000,0.344770,0.386853,0.000000,0.000000


In [266]:
#this is a file in each columns has the evaluation ndcg for each algorithm
total_results.to_csv(os.path.join(PARSE_DATA_DIR, 'total_results_k_10.csv'), index=False)